# Text Analysis and Visualization Notebook

This notebook processes a CSV file containing text data, cleans the text by removing markdown formatting and URLs, filters out non-Tagalog or Taglish text based on a specified confidence threshold, and visualizes the results. The goal is to identify and analyze Filipino phrases in the dataset.


In [ ]:
# Import necessary libraries
import pandas as pd
from lingua import Language, LanguageDetectorBuilder  # Placeholder for actual import if using a different library
from markdown import Markdown
from io import StringIO
import re
import matplotlib.pyplot as plt
import emoji
from tqdm.auto import tqdm # Import tqdm for fancy progress bars

## Configuration

Set the path to the CSV file and the threshold for identifying Tagalog content.

In [ ]:
# Configuration variables
CSV_FILENAME = 'data-Philippines-20240214-225413-hot.csv'  # Path to the CSV file
TAGALOG_THRESHOLD = 0.95  # Threshold for Tagalog content detection

# Initialize the language detector with the specified languages
languages = [Language.ENGLISH, Language.TAGALOG]

# Ensure the Tagalog threshold is within the valid range
if not (0.00 <= TAGALOG_THRESHOLD <= 1.00):
    print("ERROR: Tagalog threshold must be between 0.0 and 1.0")
    exit(1)

## Initialization

In [ ]:
detector = LanguageDetectorBuilder.from_languages(*languages).build()

## Text Processing Functions

Define functions to clean the text data by removing markdown formatting, URLs, and Reddit usernames.

In [ ]:
# Function to convert markdown formatted text to plain text
def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()

# Patching the Markdown library to add a plain text output format
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False

# Wrapper function to convert markdown text to plain text
def unmark(text):
    return __md.convert(text)

# Function to remove URLs from a text string and replace them with '[LINK]'
def remove_urls(text):
    url_pattern = re.compile(r'http\S+', re.IGNORECASE)
    return url_pattern.sub('[LINK]', text)

# Function to remove Reddit usernames from a text string and replace them with '[USERNAME]'
def remove_usernames(text):
    username_pattern = re.compile(r"/?u/[A-Za-z0-9_-]+", re.IGNORECASE)
    return username_pattern.sub('[USERNAME]', text)

# Function to remove emojis using the emoji python library
def remove_emojis(text):
    return emoji.replace_emoji(text, '')

# Replace escape sequences with a space
def remove_escape_sequences(text):
    escape_pattern = re.compile(r'[\r\n\t]', re.IGNORECASE)
    return escape_pattern.sub(' ', text)

## Main Data Processing & Saving Processed Data

Read the CSV file, process the text data, filter based on the Tagalog content threshold, and count the number of Filipino phrases identified. Save the filtered and processed data to a new CSV file.


In [ ]:
# Read the CSV file
try:
    csv_data = pd.read_csv(CSV_FILENAME, lineterminator='\n', header=0, names=[
        'submission_name',
        'submission_text',
        'body',
    ])
except FileNotFoundError:
    print("ERROR: File not found")
    exit(1)

In [ ]:
csv_data

## Count Before Filtering
Start by capturing the count of rows in your DataFrame before applying any filters:

In [ ]:
# Count the number of rows before filtering
count_before_filtering = len(csv_data)

print(f"Count before filtering: {count_before_filtering}")

In [ ]:
# Add a blank 'label' column for annotation
csv_data['label'] = ''

csv_data

In [ ]:
filipino_phrases = 0  # Counter for Filipino phrases

In [ ]:
for row in tqdm(csv_data.itertuples(), total=count_before_filtering, desc="LINES"):  # Iterate over each row in the CSV
    text = str(row.body)  # Convert to string to handle non-string data

    # Clean the text by removing markdown, URLs, usernames, emojis, 
    # and trailing whitespace
    text = unmark(text)
    text = remove_urls(text)
    text = remove_usernames(text)
    text = remove_emojis(text)
    text = remove_escape_sequences(text)
    text = text.rstrip()

    # Compute the confidence of the text being Tagalog
    result = detector.compute_language_confidence(text, Language.TAGALOG)
    
    # If the confidence is above the threshold, consider it a Filipino phrase
    if result >= TAGALOG_THRESHOLD:
        filipino_phrases += 1
        csv_data.at[row.Index, 'body'] = text  # Update the text in the CSV
    else:
        csv_data.drop(row.Index, inplace=True)  # Drop rows that don't meet the threshold

print("Finished cleaning and filtering")

In [ ]:
print(f"Total: {filipino_phrases} / {count_before_filtering}")  # Print the total number of Filipino phrases found

In [ ]:
# Save the filtered CSV
filtered_filename = CSV_FILENAME.replace('.csv', '-filtered.csv')
csv_data.to_csv(filtered_filename, index=False)  # Prevent index from being saved as a column

print(f"Filtered data saved to {filtered_filename}")

## Visualization of Filtering Effect

To understand the impact of our filtering based on the Tagalog content threshold, we will visualize the number of comments in the dataset before and after the filtering process. This will help us gauge the extent of data reduction and ensure that the filtering aligns with our expectations.

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(['Before Filtering', 'After Filtering'], [count_before_filtering, filipino_phrases], color=['blue', 'green'])
plt.title('Effect of Filtering on Dataset Size')
plt.ylabel('Number of Comments')
plt.show()